In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic-english-translation/__results__.html
/kaggle/input/arabic-english-translation/__notebook__.ipynb
/kaggle/input/arabic-english-translation/__output__.json
/kaggle/input/arabic-english-translation/custom.css
/kaggle/input/cleaned-data/cleaned_data.xlsx


In [2]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install sacrebleu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 891.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 979.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 kB 1.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from datasets import Dataset, load_metric

from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
import evaluate
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer

import warnings
warnings.filterwarnings("ignore")

2024-03-27 05:59:39.383930: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-27 05:59:39.384050: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-27 05:59:39.545722: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Dataset

In [4]:
df=pd.read_excel("/kaggle/input/cleaned-data/cleaned_data.xlsx")
df.head()

,Unnamed: 0,egyption_Text,english_Text
0,0,الأمير الصغير,The little prince
1,1,أنطوان دي سانت إكزوبيري,Antoine De Saint-Exuper
2,2,الفصل الأول,Chapter 1
3,3,في مره، لما كان عندي ست سنين، شفت صوره روعه، ف...,Once when I was six years old I saw a magnific...
4,4,وكانت عباره عن تعبان من نوع البوا بيبلع فهد.,It was a picture of a boa constrictor in the a...


In [5]:
df=df[["egyption_Text","english_Text"]]


In [6]:
train_ds = Dataset.from_pandas(df)
split_datasets = train_ds.train_test_split(train_size=0.9, seed=20)
split_datasets["validation"] = split_datasets.pop("test")

## Helsinki-NLP/opus-mt-en-ar
Source: https://huggingface.co/Helsinki-NLP/opus-mt-en-ar

In [7]:
model_checkpoint = "Helsinki-NLP/opus-mt-ar-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/917k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

In [8]:

def preprocess_function(examples):
    inputs = [ex for ex in examples["egyption_Text"]]
    targets = [ex for ex in examples["english_Text"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    # Prepare labels for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [10]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 20419
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2269
    })
})

In [11]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

pytorch_model.bin:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [12]:
!pip install -U nltk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.6 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.8.1 which is incompatible.


In [13]:
from nltk.translate.meteor_score import meteor_score


In [14]:
metrics = evaluate.combine(["sacrebleu", "meteor"])


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...


In [15]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result_bleu = metrics["sacrebleu"].compute(predictions=decoded_preds, references=decoded_labels)
    result_meteor = metrics["meteor"].compute(predictions=decoded_preds, references=decoded_labels)
    
    return {"bleu": result_bleu["score"], "meteor": result_meteor["score"]}

In [16]:
outputdir="/kaggle/working/result"

In [17]:
args = Seq2SeqTrainingArguments(
    output_dir=outputdir,
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    report_to="none",
)

In [18]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

Step,Training Loss
500,0.450700
1000,0.202700
1500,0.182100
2000,0.170700
2500,0.163300
3000,0.159200


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=3200, training_loss=0.21749749898910523, metrics={'train_runtime': 2819.4625, 'train_samples_per_second': 72.422, 'train_steps_per_second': 1.135, 'total_flos': 6921707134648320.0, 'train_loss': 0.21749749898910523, 'epoch': 10.0})

In [20]:
model_path = "/kaggle/working/saved"  # Specify your desired save path

# Save the fine-tuned model
model.save_pretrained(model_path)

# Save the tokenizer associated with the model
tokenizer.save_pretrained(model_path)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62833]], 'forced_eos_token_id': 0}


('/kaggle/working/saved/tokenizer_config.json',
 '/kaggle/working/saved/special_tokens_map.json',
 '/kaggle/working/saved/vocab.json',
 '/kaggle/working/saved/source.spm',
 '/kaggle/working/saved/target.spm',
 '/kaggle/working/saved/added_tokens.json')

In [21]:
from sklearn.model_selection import train_test_split
X = df['egyption_Text']  #  features
y = df['english_Text']  #  labels

# Splitting the dataset into training and remaining data
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.2, random_state=42)

# Splitting the remaining data into validation and test sets
X_validation, X_test, y_validation, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)



In [22]:
from transformers import pipeline
import evaluate
import pandas as pd

model_path = "/kaggle/working/saved"

translator = pipeline("translation_ar_to_en", model=model_path, tokenizer=model_path)

# Adjusted function to work with Pandas Series
def evaluate_model(X_test, y_test, translator):
    # Translate the texts in the test set
    translations = X_test.apply(lambda x: translator(x, max_length=512)[0]['translation_text'])
    
    references = [[ref] for ref in y_test]  # Evaluate expects a list of lists for references
    predictions = translations.tolist()

    bleu = evaluate.load('bleu')
    meteor = evaluate.load('meteor')

    # Compute metrics
    bleu_result = bleu.compute(predictions=predictions, references=references)
    meteor_result = meteor.compute(predictions=predictions, references=references)

    return {"bleu": bleu_result, "meteor": meteor_result}

evaluation_results = evaluate_model(X_test, y_test, translator)
print("Evaluation Results:", evaluation_results)


[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Evaluation Results: {'bleu': {'bleu': 0.23122804766498703, 'precisions': [0.5363575418994413, 0.29334527006863625, 0.18689388418633332, 0.11895649955351448], 'brevity_penalty': 0.9507937655653568, 'length_ratio': 0.9519656228727025, 'translation_length': 22375, 'reference_length': 23504}, 'meteor': {'meteor': 0.5000897235206059}}


In [23]:
model_name="/kaggle/working/result/checkpoint-3200"

In [24]:
translator = pipeline("translation", model=model_name)
translator("يارب بس ما تطلعليش سمكه كبيره أوي ويطلع كلامك على فشوش .")

[{'translation_text': "God, don't let a big fish out of your mouth."}]

In [25]:
translator("هو عاوز اي .")

[{'translation_text': 'He wants what?'}]

In [26]:
translator("الكلام علي اي يا شباب .")

[{'translation_text': "I don't know what to talk about, guys."}]

In [27]:
translator("أنا لازم أمشي.")

[{'translation_text': 'I have to go.'}]

In [28]:
translator("تشكر متحرمش ")

[{'translation_text': "Thank you. Don't be shy."}]

In [29]:
translator("عارف بيقولك ايه إذا أخطأت فأحسن")

[{'translation_text': "I know what he says. If I make a mistake, I'll do better."}]

In [30]:
translator("و ربنا احنا محتاجين فرعون")

[{'translation_text': 'God, we need a Pharaoh.'}]

In [31]:
os.mkdir("/kaggle/working/result2")

In [32]:
outputdir="/kaggle/working/result2"

In [33]:
import pickle
with open('/kaggle/working/tokenized_datasets.pickle', 'wb') as f:
    pickle.dump(tokenized_datasets, f)